In [1]:
import os
import glob
import shutil
from joblib import Parallel, delayed
import multiprocessing

In [2]:
import logging

logging.basicConfig(filename='imf-pdf2txt-data-migration.log', format='%(asctime)s - %(message)s', level=logging.ERROR)
logger = logging.getLogger('imf-pdf2text-data-migration')

In [3]:
source_dir = '/home/wb329102/R-DRIVE/Corpus/IMF/pdf_txt'
target_dir = '/home/wb329102/R-DRIVE/NLP/CORPUS/IMF/PDF_TXT'

In [4]:
def copy_file(source, target):
    shutil.copy(source, target)
    logger.error(f'Finished copying {source}')
    
    return 0

In [8]:
fnames = glob.iglob(os.path.join(source_dir, '*.pdf.txt'))
logger.error('Start processing txt files...')

workers = 100
tasks = {}

for fname in fnames:
    fid = os.path.basename(fname).replace('.pdf.txt', '')
    ofname = os.path.join(target_dir, f'imf_{fid}.txt')

    if os.path.isfile(ofname):
        logger.error(f'Document {os.path.basename(fname)} already processed...')
        continue

    if len(tasks) < workers:
        tasks[fname] = ofname
    else:
        processes = []
        for input_fname in tasks:
            p = multiprocessing.Process(target=shutil.copy, args=(input_fname, tasks[input_fname]))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()

        tasks = {}

if tasks:
    processes = []
    for input_fname in tasks:
        p = multiprocessing.Process(target=shutil.copy, args=(input_fname, tasks[input_fname]))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    tasks = {}